# 数学建模笔记

这个数学建模笔记是基于华中科技大学马世拓老师的课程（[数学建模导论：基于Python语言]( https://b23.tv/nSu6Jah)）进行记录的，希望大家能给马老师多多三连支持一下

## 线性规划模型
### 1.1 线性规划模型
线代的基础知识，这个没什么好说的
线性规划的标准形式：
$$
    \min_X C^TX\quad s.t.\begin{cases} Ax\leq b\\ AeqX=beq\\ lb \leq X\leq ub\end{cases}
$$
解决方法：
- 单纯形法
固定变量，不断变换基向量求方程组的解带入，看是不是最优解，如果不是就继续更行迭代现阶段的解
- 蒙特卡洛法
在可行域范围内生成大批量随机数据点，观察这些数据点在什么位置取得近似最优，更适用于解决非线性问题
- 引入松弛变量：不等式约束变为等式约束，解决绝对值问题，不等式过多甚至到了非线性
$$
        \max z=x_1+2x_2 \quad s.t.\begin{cases}-3x_1+2x_2\leq3\\ x_1\leq2\\ x_1-x_2\leq1,\\ x_1x_2\geq0\end{cases}\Rightarrow\begin{cases}x_3=3x_1-2x_2+3\\ x_4=-x_1-x_2+2\leq2\\ x_5=1-x_1+x_2\leq1,\\ x_1,x_2,x_3,x_4,x_5\geq0\end{cases}
$$

例题：$\max z=2x_1+3x_2-5x_3$
$$
s.t.\begin{cases}x_1+x_2+x_3=7\\ 2x_1-5x_2+x_3 \geq10\\ x_1+3x_2+x_3\leq 12\\ x_1,x_2,x_3\geq 0\end{cases}
$$


In [7]:
from scipy import optimize
import numpy as np

c = np.array([2,3,-5])
A = np.array([[-2,5,-1],[1,3,1]])
b = np.array([-10,12])
Aeq = np.array([[1,1,1]])
beq = np.array([7])
x1, x2, x3 = (0,None), (0,None), (0,None)
"""
    Linear programming: minimize a linear objective function subject to linear
    equality and inequality constraints.

    Linear programming solves problems of the following form:

    .. math::

        \min_x \ & c^T x \\
        \mbox{such that} \ & A_{ub} x \leq b_{ub},\\
        & A_{eq} x = b_{eq},\\
        & l \leq x \leq u ,

    where :math:`x` is a vector of decision variables; :math:`c`,
    :math:`b_{ub}`, :math:`b_{eq}`, :math:`l`, and :math:`u` are vectors; and
    :math:`A_{ub}` and :math:`A_{eq}` are matrices.

    Alternatively, that's:

        - minimize ::

            c @ x

        - such that ::

            A_ub @ x <= b_ub
            A_eq @ x == b_eq
            lb <= x <= ub

    Note that by default ``lb = 0`` and ``ub = None``. Other bounds can be
    specified with ``bounds``.

    Parameters
    ----------
    c : 1-D array
        The coefficients of the linear objective function to be minimized.
    A_ub : 2-D array, optional
        The inequality constraint matrix. Each row of ``A_ub`` specifies the
        coefficients of a linear inequality constraint on ``x``.
    b_ub : 1-D array, optional
        The inequality constraint vector. Each element represents an
        upper bound on the corresponding value of ``A_ub @ x``.
    A_eq : 2-D array, optional
        The equality constraint matrix. Each row of ``A_eq`` specifies the
        coefficients of a linear equality constraint on ``x``.
    b_eq : 1-D array, optional
        The equality constraint vector. Each element of ``A_eq @ x`` must equal
        the corresponding element of ``b_eq``.
    bounds : sequence, optional
        A sequence of ``(min, max)`` pairs for each element in ``x``, defining
        the minimum and maximum values of that decision variable.
        If a single tuple ``(min, max)`` is provided, then ``min`` and ``max``
        will serve as bounds for all decision variables.
        Use ``None`` to indicate that there is no bound. For instance, the
        default bound ``(0, None)`` means that all decision variables are
        non-negative, and the pair ``(None, None)`` means no bounds at all,
        i.e. all variables are allowed to be any real.
    method : str, optional
        The algorithm used to solve the standard form problem.
        :ref:`'highs' <optimize.linprog-highs>` (default),
        :ref:`'highs-ds' <optimize.linprog-highs-ds>`,
        :ref:`'highs-ipm' <optimize.linprog-highs-ipm>`,
        :ref:`'interior-point' <optimize.linprog-interior-point>` (legacy),
        :ref:`'revised simplex' <optimize.linprog-revised_simplex>` (legacy),
        and
        :ref:`'simplex' <optimize.linprog-simplex>` (legacy) are supported.
        The legacy methods are deprecated and will be removed in SciPy 1.11.0.
"""
optimize.linprog(-c,A,b,Aeq,beq,method='highs',bounds=(x1,x2,x3))

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -14.571428571428571
              x: [ 6.429e+00  5.714e-01  0.000e+00]
            nit: 3
          lower:  residual: [ 6.429e+00  5.714e-01  0.000e+00]
                 marginals: [ 0.000e+00  0.000e+00  7.143e+00]
          upper:  residual: [       inf        inf        inf]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00]
          eqlin:  residual: [ 0.000e+00]
                 marginals: [-2.286e+00]
        ineqlin:  residual: [ 0.000e+00  3.857e+00]
                 marginals: [-1.429e-01 -0.000e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0

### 1.2 非线性规划模型
目标函数非线性，约束条件非线性，如：
$$
    \min f=2x_1^2+3x_1x_3-x_2^2\quad s.t.\begin{cases}x_1^2-2x_2+3x_3=4\\ x_1+x_2-x_3\leq6\\ 2x_1-x_2+x_3\leq15 \\ x_1,x_2,x_3\geq0\end{cases}
$$
补充知识：多元函数相关知识，不多赘述

KKT条件：详情可以参考Blog:<a href="https://zhuanlan.zhihu.com/p/38163970">Karush-Kuhn-Tucker (KKT)条件</a>
将不等式约束包含进去

例题：三台火电机组的运行成本(单位：t/h)与出力限制(单位：MW)分别如下：
$$\begin{aligned}F_{\mathrm{G1}}&=4+0.3P_{\mathrm{Gl}}+0.0007P_{\mathrm{Gl}}^2,100\leqslant P_{\mathrm{Gl}}\leqslant200\\ F_{\mathrm{G2}}&=3+0.32P_{\mathrm{G2}}+0.0004P_{\mathrm{G2}}^2,120\leqslant P_{\mathrm{G2}}\leqslant250\\ F_{\mathrm{G3}}&=3.5+0.3P_{\mathrm{G3}}+0.00045P_{\mathrm{G3}}^2,150\leqslant P_{\mathrm{G3}}\leqslant300\end{aligned}
$$
当负荷为 700MW 时,求经济调度的结果。

In [1]:
from scipy import optimize
import numpy as np

def f(x):
    return 10.5+0.3*x[0]+0.32*x[1]+0.3*x[2]+0.0007*x[0]**2+0.0004*x[1]**2+0.00045*x[2]**2
constraint = ({'type':'eq', 'fun':lambda x: x[0]+x[1]+x[2]-700},{'type':'ineq', 'fun':lambda x: x[0]**2+x[1]**2+x[2]**2})
"""
    fun->迭代函数
    method->迭代方法
    x0->初值
    bounds->自变量范围
    constraints->限制条件,格式({'type':'eq' or 'ineq','fun': lambda x:...},{})
    不等式对应lambda表达式返回值大于0
"""
optimize.minimize(fun=f, method='L-BFGS-B',x0=np.array([100,120,150]),bounds=((100,200), (120,250), (150, 300)), constraints=constraint)

D:\Anaconda\envs\dl\Lib\site-packages\scipy\optimize\_minimize.py:576: RuntimeWarning: Method L-BFGS-B cannot handle constraints.
  warn('Method %s cannot handle constraints.' % method,


  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: 146.785
        x: [ 1.000e+02  1.200e+02  1.500e+02]
      nit: 0
      jac: [ 4.400e-01  4.160e-01  4.350e-01]
     nfev: 4
     njev: 1
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>

In [16]:
"""遗传算法"""
from sko.GA import GA
def f(x):
    return 10.5+0.3*x[0]+0.32*x[1]+0.3*x[2]+0.0007*x[0]**2+0.0004*x[1]**2+0.00045*x[2]**2

constraint = [lambda x: x[0]+x[1]+x[2]-700]
"""
func : function
        The func you want to do optimal
    n_dim : int
        number of variables of func
    lb : array_like
        The lower bound of every variables of func
    ub : array_like
        The upper bound of every vaiiables of func
    constraint_eq : list
        equal constraint
    constraint_ueq : list
        unequal constraint
    precision : array_like
        The precision of every vaiiables of func
    size_pop : int
        Size of population
    max_iter : int
        Max of iter
    prob_mut : float between 0 and 1
        Probability of mutation
        """
ga = GA(func=f, n_dim=3,size_pop=500,max_iter=500,constraint_eq=constraint,lb=[100,120,150],ub=[200,250,300])
ga.run()

(array([170.67217172, 233.7993253 , 295.52850297]), array([306.73282268]))

#### 案例1 
某公司有6个建筑工地要开工，每个工地的位置(用平面坐标系a,b表示，距离单位：千米)及水泥日用 量d(吨)由下表给出。规划设立两个料场位于A，B,日储量各为20吨。假设从料场到工地之间均有 直线道路相连，试确定料场的位置，并制定每天的供应计划，即从A，B两料场分别向各工地运送多 少吨水泥，使总的吨千米数最小。
![](./image/1.2.1.png)
#### 案例2
某单位领导在考虑本单位职工的升级调资方案时，要求相关部门遵守以下的规定： 
 (1) 年工资总额不超过1500000元；
 (2) 每级的人数不超过定编规定的人数；
 (3)II、III级的升级面尽可能达到现有人数的20%,
 (4) III级不足编制的人数可录用新职工，又I级的职工中有10%的人要退休.相关资料汇总于表中，试为单位领导拟定一个满足要求的调资方案。
![](./image/1.2.2.png)

In [16]:
# 案例1

In [17]:
# 案例2

### 1.3整数规划模型
纯整数规划，混合整数规划，0-1规划
#### 指派问题
假设n个人恰好做n项工作，第i个人做第j项工作的效率为$c_\mathrm{ij}$ 20, 应指派哪个人完成哪项任务，使完成效率最高。
决策变量： $x_{ij}=\begin{cases}1, 指派第$i$人完成第$j$项任务0\\0不指派第$i$人完成第$j$项任务\end{cases}$
目标函数:$\min z = C\textcircled{\cdot}X$
约束条件：$\begin{cases}\sum_ix_{ij}=1,j=1,2,\ldots,n\\ \sum_jx_{ij}=1,i=1,2,\ldots,n\\ x_{ij}=1\text{或}0\end{cases}$
解决方法
- 分支定界法
- 匈牙利法

例题；为了便于对模型进行求解与分析，假设有 4 件事 4 个人去做，各变里对应的数据假设如表1。
![](./image/1.3.1.png)

In [23]:
import scipy
from scipy.optimize import linear_sum_assignment
import numpy as np
cost = np.array([[25,29,31,42], [39,38,26,20], [34,27,28,40], [24,42,36,23]])
"""
 Parameters
    ----------
    cost_matrix : array
        The cost matrix of the bipartite graph.
    
    maximize : bool (default: False)
        Calculates a maximum weight matching if true.
    
    Returns
    -------
    row_ind, col_ind : array
        An array of row indices and one of corresponding column indices giving
        the optimal assignment. The cost of the assignment can be computed
        as ``cost_matrix[row_ind, col_ind].sum()``. The row indices will be
        sorted; in the case of a square cost matrix they will be equal to
        ``numpy.arange(cost_matrix.shape[0])``.
"""
cost[linear_sum_assignment(cost)].sum()

101

### 1.4动态规划模型
如经典的汉诺塔问题，背包问题

### 1.5贪心策略